# 메모리를 가진 계층적 에이전트 챗봇

이 예제에서는 **Checkpointer**를 사용하여 대화 상태를 유지하는 챗봇을 구현합니다.

## 구조
- **Supervisor**: 사용자 요청을 분석하여 적절한 팀에 라우팅
- **Research Team**: 웹 검색 및 정보 수집
- **Writing Team**: 문서 작성 및 편집
- **Checkpointer**: 대화 히스토리 유지 (thread_id별)

## 일반 그래프 vs 메모리 그래프

| 구분 | 일반 그래프 | 메모리 그래프 |
|------|------------|---------------|
| 상태 유지 | 호출 끝나면 휘발 | thread_id별 영구 유지 |
| 대화 연속성 | 매번 새로 시작 | 이전 대화 기억 |
| 용도 | 단발성 작업 | 챗봇, 멀티턴 대화 |

## 1. 환경 설정

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

OLLAMA_URL = os.getenv("OLLAMA_URL", "http://localhost:11434")
MODEL_NAME = "gpt-oss:20b"

print(f"Ollama URL: {OLLAMA_URL}")
print(f"Model: {MODEL_NAME}")

if os.getenv("LANGCHAIN_API_KEY"):
    print(f"LangSmith: 활성화 ({os.getenv('LANGCHAIN_PROJECT', 'default')})")
else:
    print("LangSmith: 비활성화")

## 2. LLM 및 도구 설정

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.tools import tool

llm = ChatOllama(
    model=MODEL_NAME,
    base_url=OLLAMA_URL,
    temperature=0,
)

# 간단한 시뮬레이션 도구들 (실제 API 대신)
@tool
def search_web(query: str) -> str:
    """웹에서 정보를 검색합니다."""
    # 시뮬레이션된 검색 결과
    results = {
        "ai": "AI(인공지능)는 기계가 인간의 지능을 모방하는 기술입니다. 머신러닝, 딥러닝, 자연어처리 등이 포함됩니다.",
        "python": "Python은 1991년 귀도 반 로섬이 만든 프로그래밍 언어입니다. 간결한 문법과 풍부한 라이브러리가 특징입니다.",
        "langgraph": "LangGraph는 LangChain 팀이 만든 라이브러리로, 상태 기반 멀티에이전트 워크플로우를 구축할 수 있습니다.",
    }
    for key, value in results.items():
        if key in query.lower():
            return value
    return f"'{query}'에 대한 검색 결과: 일반적인 정보를 찾았습니다."

@tool
def write_document(title: str, content: str) -> str:
    """문서를 작성합니다."""
    return f"문서 '{title}'이(가) 작성되었습니다.\n\n내용:\n{content}"

@tool
def summarize_text(text: str) -> str:
    """텍스트를 요약합니다."""
    # 간단한 시뮬레이션
    if len(text) > 100:
        return text[:100] + "... (요약됨)"
    return text

print("도구 정의 완료:", [search_web.name, write_document.name, summarize_text.name])

## 3. 상태 및 Supervisor 정의

In [ ]:
from typing import Literal
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.types import Command
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, AIMessage


class State(MessagesState):
    """대화 상태 - messages는 Checkpointer에 의해 유지됨"""
    next: str


def make_supervisor_node(llm, members: list[str]):
    """Supervisor 노드 생성"""
    options = ["FINISH"] + members
    system_prompt = (
        "You are a supervisor tasked with managing a conversation between the"
        f" following workers: {members}. Given the following user request,"
        " respond with the worker to act next. Each worker will perform a"
        " task and respond with their results and status. When finished,"
        " respond with FINISH."
        f"\n\nYou must respond with ONLY one of these options: {options}"
    )

    class Router(BaseModel):
        """Worker to route to next. If no workers needed, route to FINISH."""
        next: str = Field(description=f"Next worker to route to. Must be one of: {options}")

    def supervisor_node(state: State) -> Command[Literal[*members, "__end__"]]:
        """LLM 기반 라우터."""
        messages = [
            {"role": "system", "content": system_prompt},
        ] + state["messages"]
        
        response = llm.with_structured_output(Router).invoke(messages)
        goto = response.next
        if goto == "FINISH":
            goto = END

        return Command(goto=goto, update={"next": goto})

    return supervisor_node

print("Supervisor 정의 완료")

## 4. 팀 에이전트 정의

In [ ]:
# Research Team
research_agent = create_react_agent(
    llm, 
    tools=[search_web],
    prompt="당신은 정보를 검색하고 조사하는 리서치 전문가입니다. 사용자의 질문에 대해 검색하고 답변하세요."
)

def research_node(state: State) -> Command[Literal["supervisor"]]:
    result = research_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="research_team")
            ]
        },
        goto="supervisor",
    )


# Writing Team
writing_agent = create_react_agent(
    llm,
    tools=[write_document, summarize_text],
    prompt="당신은 문서 작성 전문가입니다. 요청에 따라 문서를 작성하거나 텍스트를 요약하세요."
)

def writing_node(state: State) -> Command[Literal["supervisor"]]:
    result = writing_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="writing_team")
            ]
        },
        goto="supervisor",
    )

print("팀 에이전트 정의 완료 (HumanMessage 사용)")

## 5. 그래프 구성 (with Checkpointer)

**핵심**: `MemorySaver`를 사용하여 대화 상태를 유지합니다.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# Checkpointer 생성 - 메모리에 상태 저장
memory = MemorySaver()

# Supervisor 노드 생성
supervisor_node = make_supervisor_node(llm, ["research_team", "writing_team"])

# 그래프 구성
builder = StateGraph(State)
builder.add_node("supervisor", supervisor_node)
builder.add_node("research_team", research_node)
builder.add_node("writing_team", writing_node)

builder.add_edge(START, "supervisor")

# Checkpointer와 함께 컴파일 - 이것이 핵심!
chatbot = builder.compile(checkpointer=memory)

print("챗봇 그래프 컴파일 완료 (with MemorySaver)")

In [ ]:
from IPython.display import Image, display

display(Image(chatbot.get_graph().draw_mermaid_png()))

## 6. 챗봇 대화 함수

In [ ]:
def chat(message: str, thread_id: str = "default"):
    """
    챗봇과 대화합니다.
    
    Args:
        message: 사용자 메시지
        thread_id: 대화 스레드 ID (같은 ID면 대화 이어짐)
    """
    config = {"configurable": {"thread_id": thread_id}}
    
    print(f"\n{'='*60}")
    print(f"[사용자] {message}")
    print(f"{'='*60}")
    
    # 스트리밍으로 실행
    for event in chatbot.stream(
        {"messages": [("user", message)]},
        config,
        stream_mode="updates"
    ):
        for node_name, node_output in event.items():
            if node_name == "supervisor":
                next_node = node_output.get("next", "unknown")
                print(f"\n[Supervisor] → {next_node}")
            elif "messages" in node_output:
                for msg in node_output["messages"]:
                    if hasattr(msg, "content") and msg.content:
                        print(f"\n[{node_name}]")
                        print(msg.content[:500] + "..." if len(msg.content) > 500 else msg.content)


def show_history(thread_id: str = "default"):
    """대화 히스토리를 확인합니다."""
    config = {"configurable": {"thread_id": thread_id}}
    state = chatbot.get_state(config)
    
    print(f"\n{'='*60}")
    print(f"Thread '{thread_id}' 대화 히스토리")
    print(f"{'='*60}")
    
    if state.values:
        messages = state.values.get("messages", [])
        for i, msg in enumerate(messages):
            role = msg.type if hasattr(msg, 'type') else 'unknown'
            name = getattr(msg, 'name', '')
            content = msg.content[:100] + "..." if len(msg.content) > 100 else msg.content
            print(f"\n[{i+1}] {role} {f'({name})' if name else ''}")
            print(f"    {content}")
    else:
        print("대화 히스토리가 없습니다.")

print("대화 함수 정의 완료")

## 7. 테스트: 대화 연속성 확인

같은 `thread_id`로 여러 번 호출하면 이전 대화를 기억합니다.

In [ ]:
# 첫 번째 대화 - AI에 대해 질문
chat("AI가 뭐야?", thread_id="user-123")

In [ ]:
# 두 번째 대화 - 이전 대화를 참조하는 후속 질문
chat("방금 설명한 내용을 문서로 정리해줘", thread_id="user-123")

In [ ]:
# 세 번째 대화 - 또 다른 후속 질문
chat("더 자세히 알려줘", thread_id="user-123")

In [ ]:
# 대화 히스토리 확인
show_history(thread_id="user-123")

## 8. 비교: 다른 thread_id는 별도 대화

다른 `thread_id`를 사용하면 새로운 대화가 시작됩니다.

In [ ]:
# 다른 사용자의 대화 (새로운 thread)
chat("Python이 뭐야?", thread_id="user-456")

In [ ]:
# user-456의 히스토리 - user-123과 별도
show_history(thread_id="user-456")

In [ ]:
# user-123의 히스토리 - 여전히 유지됨
show_history(thread_id="user-123")

## 9. 요약

### Checkpointer의 역할

```python
# 핵심 코드
memory = MemorySaver()
chatbot = builder.compile(checkpointer=memory)

# 호출 시 thread_id로 대화 구분
config = {"configurable": {"thread_id": "user-123"}}
chatbot.invoke({"messages": [...]}, config)
```

### 상태 유지 구조

```
MemorySaver
├── thread: "user-123"
│   └── State
│       ├── messages: [User, AI, User, AI, ...] (누적)
│       └── next: "research_team"
│
├── thread: "user-456"
│   └── State
│       ├── messages: [...] (별도 대화)
│       └── next: "..."
│
└── thread: "user-789"
    └── ...
```

### 프로덕션 환경

| 환경 | Checkpointer | 특징 |
|------|--------------|------|
| 개발/테스트 | `MemorySaver` | 메모리, 재시작 시 휘발 |
| 프로덕션 | `SqliteSaver` | 파일 기반, 영구 저장 |
| 대규모 | `PostgresSaver` | DB 기반, 확장성 |